In [0]:
# https://keras.io/
#!pip install -q keras
import keras

import keras
#from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D,AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,SGD
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 32
num_classes = 200
epochs = 50
l = 10
num_filter = 20

#layer counter for naming
l_count=0

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cp '/gdrive/My Drive/EIP3/session4/clr_callback.py' clr_callback.py

In [0]:
from clr_callback import *

clr = CyclicLR(base_lr=0.0001, max_lr=0.0005,
                                step_size=2000., mode='triangular2')

In [0]:
def space_to_depth_x2(x):
    import tensorflow as tf
    return tf.space_to_depth(x, block_size=4)

In [0]:
#layer counter for naming
l_count=0

def create_conv_block(x,f_num=16,numlayers=5,maxpool=False,blocknum=1):
    # you have used a block of following layers: 32 (same) > 64 (same) > 128 (same) > 256 (same) > 512 (same) > MP
    global l_count
    l_count=l_count+1
    print(x.shape)
    for i in range(1,numlayers):
      
      f_num=f_num*2
      #print(blocknum, i,f_num)
      x = Conv2D(f_num, (3,3), strides=(1,1), padding='same',name='block_'+str(blocknum)+'_conv_'+str(l_count), use_bias=False)(x)
      x = BatchNormalization(name='block_'+str(blocknum)+'_norm_'+str(l_count))(x)
      x = ReLU()(x)
      l_count=l_count+1
    if (maxpool):
      x = MaxPooling2D(pool_size=(2, 2))(x)
    
    return x
  

In [0]:
input = Input(shape=(None, None, 3))

# Block 1
block1=create_conv_block(input,32,3,False,1)
# Block 2
block2=create_conv_block(block1,64,3,False,2)

#Block3 
block3 = create_conv_block(block2,64,3,False,3)

mod_block1=Conv2D(128,1,use_bias=False)(block1)

#mod_block3=Conv2D(128,1,use_bias=False)(block3)

#concatenate block1_adjusted and block3
concat_layer1 = concatenate([mod_block1, block3])

if (concat_layer1.shape[1]>=64):
  p=3
else:
  p=2

reduce1=MaxPooling2D(pool_size=(p, p))(concat_layer1)

#Block4
block4=create_conv_block(reduce1,64,4,False,4)



#adjust the shape of block 1 to be concatenated to block4
#mod_block1=Lambda(space_to_depth_x2)(block1)



#Block5
block5=create_conv_block(block4,128,4,True,5)

block6=create_conv_block(block5,64,4,True,6)

#tail1 = Conv2D(num_classes,1,use_bias=False)(block6)



#tail2 = Conv2D(num_classes,3,use_bias=False)(tail1)

#use average pooling on this 2x2 size output 
last_layer = Conv2D(num_classes,1,use_bias=False)(block6)

# Adjust channels to required numbers and Flatten 
flat_layer=GlobalAveragePooling2D()(last_layer)
#flat_layer = Flatten()(last_layer)

#get the output using softmax on this flattened layer 

output=Activation('softmax')(flat_layer)



(?, ?, ?, 3)
Instructions for updating:
Colocations handled automatically by placer.
(?, ?, ?, 128)
(?, ?, ?, 256)
(?, ?, ?, 384)
(?, ?, ?, 512)
(?, ?, ?, 1024)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()
"""
#mount drive 
from google.colab import drive
drive.mount('/gdrive')
from keras.utils import plot_model
plot_model(model, to_file="/gdrive/My Drive/EIP3/session3/Tiny_ImgNetReLU_Data_Aug_Model.png")
"""

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block_1_conv_1 (Conv2D)         (None, None, None, 6 1728        input_1[0][0]                    
__________________________________________________________________________________________________
block_1_norm_1 (BatchNormalizat (None, None, None, 6 256         block_1_conv_1[0][0]             
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, None, None, 6 0           block_1_norm_1[0][0]             
__________________________________________________________________________________________________
block_1_co

'\n#mount drive \nfrom google.colab import drive\ndrive.mount(\'/gdrive\')\nfrom keras.utils import plot_model\nplot_model(model, to_file="/gdrive/My Drive/EIP3/session3/Tiny_ImgNetReLU_Data_Aug_Model.png")\n'

In [0]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


In [0]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
#!cp '/gdrive/My Drive/EIP3/session4/tiny-imagenet-200.zip' tiny-imagenet-200.zip

--2019-04-07 03:45:29--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  19.5MB/s    in 15s     

2019-04-07 03:45:45 (15.5 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
!unzip -qq 'tiny-imagenet-200.zip'
#!ls

In [0]:
import pandas as pd
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

In [0]:
# Use Augmentaion parameters as required.
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True)
"""
ImageDataGenerator(
        rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
        
        """

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
batch_size=256


In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(32, 32), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(32, 32),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=25, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
390/390 [==============================] - 565s 1s/step - loss: 4.7744 - acc: 0.0444 - val_loss: 5.2868 - val_acc: 0.0489
Epoch 2/25
390/390 [==============================] - 553s 1s/step - loss: 4.0482 - acc: 0.1202 - val_loss: 4.3808 - val_acc: 0.1100
Epoch 3/25
390/390 [==============================] - 544s 1s/step - loss: 3.5872 - acc: 0.1908 - val_loss: 4.4812 - val_acc: 0.1282
Epoch 4/25
390/390 [==============================] - 545s 1s/step - loss: 3.2662 - acc: 0.2492 - val_loss: 4.1739 - val_acc: 0.1453
Epoch 5/25
390/390 [==============================] - 544s 1s/step - loss: 3.0122 - acc: 0.2970 - val_loss: 3.3904 - val_acc: 0.2365
Epoch 6/25
390/390 [==============================] - 544s 1s/step - loss: 2.8045 - acc: 0.3339 - val_loss: 3.2310 - val_acc: 0.2704
Epoch 7/25
390/390 [==============================] - 545s 1s/step - loss: 2.6289 - acc: 0.3703 - val_loss: 3.1632 - val_acc: 0.2834
Epoch 8/25
390/390 [=

In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch25.h5")
model.save_weights("/gdrive/My Drive/EIP3/session4/model_weights_custom3_tiny_imgnet_epoch25.h5")

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch25.h5")

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=10*100000//batch_size

In [0]:
clr = CyclicLR(base_lr=0.0001, max_lr=0.0005,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)


valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2204s 2s/step - loss: 1.9122 - acc: 0.5149 - val_loss: 2.1302 - val_acc: 0.4823
Epoch 2/5
1041/1041 [==============================] - 2193s 2s/step - loss: 1.6017 - acc: 0.5872 - val_loss: 2.0880 - val_acc: 0.4913
Epoch 3/5
1041/1041 [==============================] - 2180s 2s/step - loss: 1.4612 - acc: 0.6209 - val_loss: 2.0512 - val_acc: 0.5037
Epoch 4/5
1041/1041 [==============================] - 2175s 2s/step - loss: 1.3532 - acc: 0.6470 - val_loss: 1.9841 - val_acc: 0.5175
Epoch 5/5
1041/1041 [==============================] - 2177s 2s/step - loss: 1.2551 - acc: 0.6704 - val_loss: 2.0383 - val_acc: 0.5095


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch30.h5")

End of epoch 30 
trained on 32x32 size with horizontal flip for 25 epochs and then on 64x64 size with no augmentation.
max validation accuracy is 51.75 at the 29th epoch 

In [0]:
##############################end of 30 epochs ##########################

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch30.h5")

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=10*100000//batch_size

In [0]:
clr = CyclicLR(base_lr=0.00005, max_lr=0.0005,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)


valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2218s 2s/step - loss: 1.1448 - acc: 0.6962 - val_loss: 2.1107 - val_acc: 0.4999


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch31.h5")

End of epoch 31

In [0]:
###################end of epoch 31 ###############

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2212s 2s/step - loss: 1.0677 - acc: 0.7155 - val_loss: 2.0446 - val_acc: 0.5110


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch32.h5")

end of epoch 32

In [0]:
########### end of epoch 32 #########

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch32.h5")

In [0]:
num_steps_for_clr=6*100000//batch_size
clr = CyclicLR(base_lr=0.00005, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)


valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2223s 2s/step - loss: 0.8019 - acc: 0.7993 - val_loss: 1.9457 - val_acc: 0.5288


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch33.h5")

end of epoch 33

In [0]:
######### end of epoch 33 #########

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch33.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=256
img_size=32
num_steps_for_clr=10*100000//batch_size

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=[0.7,1.7])


valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00005, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
390/390 [==============================] - 587s 2s/step - loss: 2.9690 - acc: 0.4123 - val_loss: 3.6551 - val_acc: 0.3807


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch34.h5")

End of epoch 34 

In [0]:
################## end of epoch 34 - applied zoom augmentation on 32 x 32 images #####################

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
390/390 [==============================] - 577s 1s/step - loss: 2.1236 - acc: 0.4891 - val_loss: 3.2498 - val_acc: 0.3787


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch35.h5")

end of epoch 35 - trained on 32x32 images with zoom augmentation

In [0]:
######### end of epoch 35 ############

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.01,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=[0.3,1.20])


valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2280s 2s/step - loss: 2.6883 - acc: 0.3729 - val_loss: 4.0706 - val_acc: 0.2537


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch36.h5")

end of epoch 36 . trained 64x64 images with zoom augmentation

In [0]:
###############end of epoch 36 

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2290s 2s/step - loss: 2.8008 - acc: 0.3478 - val_loss: 4.3192 - val_acc: 0.1904


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch37.h5")

end of epoch 37 . trained 64 x64 image with zoom augmentation

In [0]:
####### end of epoch 37 

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=45)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2290s 2s/step - loss: 2.5246 - acc: 0.3939 - val_loss: 5.0812 - val_acc: 0.1710


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch38.h5")

end of epoch 38 . trained 64 x64 images with rotation augmentation

In [0]:
#end of epoch 38 

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.01,
                                step_size=num_steps_for_clr, mode='triangular2')

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 10000 images belonging to 200 classes.


In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch38.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
model.fit_generator(train_generator, epochs=7, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/7
1041/1041 [==============================] - 2290s 2s/step - loss: 1.6549 - acc: 0.5820 - val_loss: 2.1372 - val_acc: 0.4840
Epoch 2/7
1041/1041 [==============================] - 2272s 2s/step - loss: 1.6784 - acc: 0.5691 - val_loss: 3.6941 - val_acc: 0.2827
Epoch 3/7
1041/1041 [==============================] - 2270s 2s/step - loss: 1.7964 - acc: 0.5424 - val_loss: 3.0474 - val_acc: 0.3337
Epoch 4/7
1041/1041 [==============================] - 2271s 2s/step - loss: 1.8037 - acc: 0.5404 - val_loss: 2.6646 - val_acc: 0.3986
Epoch 5/7
1041/1041 [==============================] - 2272s 2s/step - loss: 1.7580 - acc: 0.5524 - val_loss: 3.8331 - val_acc: 0.2858
Epoch 6/7
1041/1041 [==============================] - 2272s 2s/step - loss: 1.6991 - acc: 0.5641 - val_loss: 3.6634 - val_acc: 0.3055
Epoch 7/7
1041/1041 [==============================] - 2278s 2s/step - loss: 1.6310 - acc: 0.5783 - val_loss: 2.7143 - val_acc: 0.4101


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch45.h5")

end of epoch 45 . trained 7 epochs on 64x64 images with no augmentation

In [0]:
######### end of epoch 45

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch45.h5")

In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2277s 2s/step - loss: 1.1814 - acc: 0.6922 - val_loss: 1.7539 - val_acc: 0.5690
Epoch 2/5
1041/1041 [==============================] - 2269s 2s/step - loss: 0.9052 - acc: 0.7638 - val_loss: 1.6895 - val_acc: 0.5836
Epoch 3/5
1041/1041 [==============================] - 2254s 2s/step - loss: 0.7664 - acc: 0.7968 - val_loss: 1.7154 - val_acc: 0.5898
Epoch 4/5
1041/1041 [==============================] - 2245s 2s/step - loss: 0.6594 - acc: 0.8243 - val_loss: 1.7283 - val_acc: 0.5925
Epoch 5/5
1041/1041 [==============================] - 2228s 2s/step - loss: 0.5617 - acc: 0.8500 - val_loss: 1.7680 - val_acc: 0.5927


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch50.h5")

end of epoch 50 .trained for 5 epochs with 64x64 images -no augmentation .**Max val accuracy reached at epoch 50 - 59.27 **

In [0]:
########## end of epoch 50 -val accuracy 59.27  ############

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
1041/1041 [==============================] - 2219s 2s/step - loss: 0.4692 - acc: 0.8749 - val_loss: 1.8967 - val_acc: 0.5856


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch51.h5")

End of epoch 51 .val accuracy came down from 59.27 to 58.56 . Add more augmentation 

In [0]:
###### end of epoch 51 

In [0]:
batch_size=256
img_size=32
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=15.0)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
390/390 [==============================] - 579s 1s/step - loss: 3.4634 - acc: 0.4235 - val_loss: 3.8690 - val_acc: 0.3945
Epoch 2/5
390/390 [==============================] - 571s 1s/step - loss: 2.5293 - acc: 0.4770 - val_loss: 3.0713 - val_acc: 0.4155
Epoch 3/5
390/390 [==============================] - 561s 1s/step - loss: 2.0374 - acc: 0.5073 - val_loss: 2.5829 - val_acc: 0.4391
Epoch 4/5
390/390 [==============================] - 563s 1s/step - loss: 1.8284 - acc: 0.5383 - val_loss: 2.4900 - val_acc: 0.4488
Epoch 5/5
390/390 [==============================] - 563s 1s/step - loss: 1.6948 - acc: 0.5651 - val_loss: 2.4112 - val_acc: 0.4609


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch56.h5")

In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
390/390 [==============================] - 564s 1s/step - loss: 1.5773 - acc: 0.5896 - val_loss: 2.3791 - val_acc: 0.4656
Epoch 2/5
390/390 [==============================] - 563s 1s/step - loss: 1.4792 - acc: 0.6128 - val_loss: 2.3926 - val_acc: 0.4659
Epoch 3/5
390/390 [==============================] - 562s 1s/step - loss: 1.3695 - acc: 0.6376 - val_loss: 2.3962 - val_acc: 0.4722
Epoch 4/5
390/390 [==============================] - 563s 1s/step - loss: 1.2582 - acc: 0.6628 - val_loss: 2.3626 - val_acc: 0.4837
Epoch 5/5
390/390 [==============================] - 563s 1s/step - loss: 1.1411 - acc: 0.6910 - val_loss: 2.4223 - val_acc: 0.4848


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch61.h5")

In [0]:
model.fit_generator(train_generator, epochs=1, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/1
390/390 [==============================] - 557s 1s/step - loss: 1.0322 - acc: 0.7186 - val_loss: 2.4103 - val_acc: 0.4906


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch62.h5")

In [0]:
model.fit_generator(train_generator, epochs=3, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/3
390/390 [==============================] - 564s 1s/step - loss: 0.9463 - acc: 0.7416 - val_loss: 2.4224 - val_acc: 0.4948
Epoch 2/3
390/390 [==============================] - 563s 1s/step - loss: 0.8695 - acc: 0.7620 - val_loss: 2.5247 - val_acc: 0.4875
Epoch 3/3
390/390 [==============================] - 563s 1s/step - loss: 0.8078 - acc: 0.7795 - val_loss: 2.5055 - val_acc: 0.4948


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch65.h5")

End of epoch 65 . Trained epochs 52-65 on 32x32 images with Shear transform augmentation 

In [0]:
######End of epoch 65 #######

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=15.0)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2258s 2s/step - loss: 1.1332 - acc: 0.6964 - val_loss: 1.8350 - val_acc: 0.5598
Epoch 2/5
1041/1041 [==============================] - 2254s 2s/step - loss: 0.7637 - acc: 0.7931 - val_loss: 1.7712 - val_acc: 0.5883
Epoch 3/5
1041/1041 [==============================] - 2253s 2s/step - loss: 0.6350 - acc: 0.8277 - val_loss: 1.8099 - val_acc: 0.5891
Epoch 4/5
1041/1041 [==============================] - 2254s 2s/step - loss: 0.5614 - acc: 0.8458 - val_loss: 1.8645 - val_acc: 0.5863
Epoch 5/5
1041/1041 [==============================] - 2252s 2s/step - loss: 0.5029 - acc: 0.8616 - val_loss: 1.8902 - val_acc: 0.5877


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch70.h5")

End of epoch 70. Trained 5 epochs with 64x64 images and augmentation shear transform .val accuracy went to 58.91 at epoch 68 .Max val accuracy so far remains 59.27 

In [0]:
#### end of epoch 70. 4 augmentations so far : horizontal_flip, zoom, rotation, shear ######

In [0]:
### do 5 epochs without augmentation at 64x64 size

batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2256s 2s/step - loss: 0.2862 - acc: 0.9332 - val_loss: 1.7454 - val_acc: 0.6168
Epoch 2/5
 977/1041 [===========================>..] - ETA: 2:14 - loss: 0.2564 - acc: 0.9413

In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch75.h5")

End of epoch 71 (Runtime disconnected after epoch 71) trained on 64x64 images with no augmentation for this epoch . 

**Max validation accuracy at the end of epoch 71 is 61.68**

In [0]:
######### end of epoch 71 ######

In [0]:
### load last saved model and do 4 more epochs without augmentation at 64x64 size

batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch70.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
from keras.callbacks import ModelCheckpoint

  
chkpoint_model=ModelCheckpoint("/gdrive/My Drive/EIP3/session4/model_custom_3_tiny_imgnet_best.h5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
chkpoint_weights=ModelCheckpoint("/gdrive/My Drive/EIP3/session4/model_weights_custom_3_tiny_imgnet_best.h5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=True, mode='max')

In [0]:
model.fit_generator(train_generator, epochs=4, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr,chkpoint_model,chkpoint_weights],verbose=1)

Epoch 1/4
1041/1041 [==============================] - 2339s 2s/step - loss: 0.2862 - acc: 0.9332 - val_loss: 1.7453 - val_acc: 0.6168
Epoch 2/4
1041/1041 [==============================] - 2329s 2s/step - loss: 0.2557 - acc: 0.9417 - val_loss: 1.7860 - val_acc: 0.6120
Epoch 3/4
1041/1041 [==============================] - 2305s 2s/step - loss: 0.2269 - acc: 0.9496 - val_loss: 1.8813 - val_acc: 0.6013
Epoch 4/4
1041/1041 [==============================] - 2299s 2s/step - loss: 0.1895 - acc: 0.9600 - val_loss: 1.9519 - val_acc: 0.5987


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch75.h5")

End of epoch 75 

In [0]:
### load last saved model and do 4 more epochs without augmentation at 32x32 size

batch_size=256
img_size=32
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, width_shift_range=0.4, height_shift_range=0.4)




valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)



Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr,chkpoint_model,chkpoint_weights],verbose=1)

Epoch 1/5
390/390 [==============================] - 592s 2s/step - loss: 6.6115 - acc: 0.2269 - val_loss: 5.4504 - val_acc: 0.3733
Epoch 2/5
390/390 [==============================] - 585s 2s/step - loss: 3.9731 - acc: 0.2536 - val_loss: 2.9669 - val_acc: 0.3905
Epoch 3/5
390/390 [==============================] - 575s 1s/step - loss: 3.2103 - acc: 0.2865 - val_loss: 2.6873 - val_acc: 0.4174
Epoch 4/5
390/390 [==============================] - 575s 1s/step - loss: 2.9941 - acc: 0.3172 - val_loss: 2.6599 - val_acc: 0.4292
Epoch 5/5
390/390 [==============================] - 576s 1s/step - loss: 2.8528 - acc: 0.3385 - val_loss: 2.6190 - val_acc: 0.4298


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch80.h5")

End of epoch 80 . added 3 more augmentations v_flip, width_shift and height_shift 

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, width_shift_range=0.4, height_shift_range=0.4)




valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr,chkpoint_model,chkpoint_weights],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2297s 2s/step - loss: 2.3523 - acc: 0.4318 - val_loss: 2.1165 - val_acc: 0.5232
Epoch 2/5
1041/1041 [==============================] - 2293s 2s/step - loss: 2.0115 - acc: 0.5019 - val_loss: 2.0905 - val_acc: 0.5513
Epoch 3/5
1041/1041 [==============================] - 2296s 2s/step - loss: 1.9044 - acc: 0.5264 - val_loss: 2.1370 - val_acc: 0.5551
Epoch 4/5
1041/1041 [==============================] - 2301s 2s/step - loss: 1.8454 - acc: 0.5377 - val_loss: 2.1975 - val_acc: 0.5433
Epoch 5/5
1041/1041 [==============================] - 2311s 2s/step - loss: 1.7830 - acc: 0.5499 - val_loss: 2.2362 - val_acc: 0.5505


In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch85.h5")

end of epoch 85 .Augmentations applied so far -horizontal flip, zoom, rotation, shear , vertical flip , width shift , height shift 

In [0]:
############end of epoch 85 

In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255)




valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=5, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr,chkpoint_model,chkpoint_weights],verbose=1)

Epoch 1/5
1041/1041 [==============================] - 2318s 2s/step - loss: 0.6243 - acc: 0.8340 - val_loss: 1.7285 - val_acc: 0.6099
Epoch 2/5
1041/1041 [==============================] - 2314s 2s/step - loss: 0.3339 - acc: 0.9164 - val_loss: 1.7490 - val_acc: 0.6172
Epoch 3/5
 392/1041 [==========>...................] - ETA: 23:20 - loss: 0.2402 - acc: 0.9475

In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch90.h5")

Session ended after epoch 87 . **max val accuracy was 61.72 at the end of epoch 87 ** . 
Load best model saved via call back and continue training for another 8 epochs to see if val accuracy improves 

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom_3_tiny_imgnet_best.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=96
img_size=64
num_steps_for_clr=8*100000//batch_size
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
                                step_size=num_steps_for_clr, mode='triangular2')

train_datagen = ImageDataGenerator(rescale=1./255)




valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_size,img_size), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)


validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
model.fit_generator(train_generator, epochs=8, 
                        steps_per_epoch=100000//batch_size, 
                    validation_steps=10000//batch_size, 
                    validation_data=validation_generator,shuffle=True,callbacks=[clr,chkpoint_model,chkpoint_weights],verbose=1)

Epoch 1/8
1041/1041 [==============================] - 2220s 2s/step - loss: 0.2366 - acc: 0.9494 - val_loss: 1.7570 - val_acc: 0.6188
Epoch 2/8
1041/1041 [==============================] - 2207s 2s/step - loss: 0.1980 - acc: 0.9611 - val_loss: 1.8087 - val_acc: 0.6174
Epoch 3/8
1041/1041 [==============================] - 2192s 2s/step - loss: 0.1569 - acc: 0.9729 - val_loss: 1.9035 - val_acc: 0.6101
Epoch 4/8
1041/1041 [==============================] - 2197s 2s/step - loss: 0.1114 - acc: 0.9845 - val_loss: 2.0051 - val_acc: 0.6018
Epoch 5/8
1041/1041 [==============================] - 2205s 2s/step - loss: 0.0912 - acc: 0.9862 - val_loss: 2.1050 - val_acc: 0.6013
Epoch 6/8
  81/1041 [=>............................] - ETA: 32:47 - loss: 0.0715 - acc: 0.9901

In [0]:
model.save("/gdrive/My Drive/EIP3/session4/model_custom3_tiny_imgnet_epoch95.h5")

session disconnected after epoch 92. **By now the model has clearly overfit . Max validation accuracy was 61.88**

In [0]:
### load best model (best val accuracy was 61.88) and evaluate to check score #####

In [0]:
model=load_model("/gdrive/My Drive/EIP3/session4/model_custom_3_tiny_imgnet_best.h5")

Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=100
img_size=64

valid_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_size,img_size),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=False, seed=42)


score=model.evaluate_generator(generator=validation_generator,steps=10000//batch_size)
print(score)

Found 10000 images belonging to 200 classes.
[1.7567966926097869, 0.6187000000476837]


**model.evaluate gives an accuracy of 61.87 which is pretty close to max validation accuracy 0f 61.88 ** 

Identifying and extra training for hard to predict classes should probably have been done at the 45-50 epoch mark where it was already starting to overfit training samples . Perhaps running more epochs with augmented sets would have yielded better results . Will have to go back to a model at around 40 epoch mark and try these improvement strategies